# Difference DDPG and TD3
## Problems of DDPG
- Q-function overestimates Q-values dramatically
- policy exploits the errors in the Q-function \n
-> policy breaking
TD3 is a direct successor of DDPG and improves it using three major tricks.

## 1. Clipped double Q-Learning

## 2. Delayed policy update

## 3. Target policy smoothing

